In [1]:
from pathlib import Path
import pandas as pd

In [2]:
useless_data = ['Missing', 'missing', 'Not available', 'not determined', 'not collected',
               'unknown', 'Not Applicable', 'not applicable', 'not known', 'not provided',
               'Not applicable', 'Not Collected']

In [3]:
run_info = pd.read_csv('/media/NGS/Data_Analysis/NCBI/Run_Info/Salmonella_enterica_Run_info.csv',
                       usecols=['Run', 'BioSample'])

In [5]:
sample_info = pd.read_csv('/media/NGS/Data_Analysis/NCBI/Sample_Info/Salmonella_enterica_Enteritidis_SampleInfo.csv',
                          low_memory=False)

In [6]:
sample_info = sample_info.merge(run_info)

In [7]:
sample_info = sample_info.replace(useless_data, pd.NA).dropna(how='all', axis=1)

In [8]:
db_mapping = {'SRR': 'NCBI', 'ERR': 'ENA', 'DRR': 'DDBJ'}

In [9]:
sample_info['Database'] = sample_info['Run'].str[:3].map(db_mapping)

In [10]:
sheet_pages = {}
for db, group in sample_info.groupby('Database'):
    group = group.dropna(axis=1, how='all').drop_duplicates('BioSample').drop(['Database', 'Run'], axis=1)
    sheet_pages[db] = group

In [11]:
with pd.ExcelWriter('/media/NGS/Data_Analysis/NCBI/Sample_Info/Salmonella_enterica_Enteritidis_SampleInfo.xlsx') as writer:
    for sheet_name, table in sheet_pages.items():
        table.to_excel(writer, sheet_name=sheet_name, index=False)